- 단계구분도: 경기도내 인구소멸비율

In [8]:
import json, folium
import numpy as np
import pandas as pd

- 경기도 시,군,구별 인구 소멸 비율 구하기

In [9]:
dis_pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = dis_pop[['행정구역구분명','행정구역명','60~69세','70~79세','80~89세','90~99세','100세 이상','20~29세 (여)','30~39세 (여)']]
dis_pop['인구소멸비율'] = (pop.iloc[:, -2:].sum(axis=1) / pop.iloc[:, 2:7].sum(axis=1)).round(2)
pop = dis_pop[['행정구역구분명','행정구역명','총 인구수','인구소멸비율']]
pop.head()

행정구역구분명        행정구역명     총 인구수  인구소멸비율
0       도        경기도    13589432    0.57
1      시군     경기도 가평군      62197    0.20
2     읍면동  경기도 가평군 가평읍     19555    0.26
3     읍면동   경기도 가평군 북면      3815    0.09
4     읍면동   경기도 가평군 상면      5669    0.11

In [10]:
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

행정구역구분명          행정구역명    총 인구수  인구소멸비율
1       시군       경기도 가평군     62197    0.20
8       시군       경기도 고양시   1075202    0.57
9        구   경기도 고양시 덕양구    487874    0.56
31       구  경기도 고양시 일산동구    296590    0.60
44       구  경기도 고양시 일산서구    290738    0.56

- geo_data

In [11]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename,encoding='utf-8'))

- 인구 데이터와 geo 데이터의 index 일치 시키기

In [12]:
pop['시군구'] = pop.행정구역명.apply(lambda x : ''.join(x.split()[1:]))
pop.head()

행정구역구분명          행정구역명    총 인구수  인구소멸비율      시군구
1       시군       경기도 가평군     62197    0.20      가평군
8       시군       경기도 고양시   1075202    0.57      고양시
9        구   경기도 고양시 덕양구    487874    0.56   고양시덕양구
31       구  경기도 고양시 일산동구    296590    0.60  고양시일산동구
44       구  경기도 고양시 일산서구    290738    0.56  고양시일산서구

In [13]:
pop.set_index('시군구',inplace=True)
pop.head()

행정구역구분명          행정구역명    총 인구수  인구소멸비율
시군구                                            
가평군          시군       경기도 가평군     62197    0.20
고양시          시군       경기도 고양시   1075202    0.57
고양시덕양구        구   경기도 고양시 덕양구    487874    0.56
고양시일산동구       구  경기도 고양시 일산동구    296590    0.60
고양시일산서구       구  경기도 고양시 일산서구    290738    0.56

In [14]:
map = folium.Map(location=[37.4,127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=pop.인구소멸비율,        #단계구분도로 보여줄 데이터
    columns=[pop.index,pop.인구소멸비율],      # 데이터 프레임에서 추출할 column
    fill_color='YlOrRd',       # Color map
    key_on='properties.name'
).add_to(map)
map
